In [ ]:
date_range = snakemake.params.date_range

In [ ]:
import logging

import atlite

logging.basicConfig(level=logging.INFO)


import geopandas as gpd
import pandas as pd
import xarray as xr
from shapely.geometry import Polygon

In [ ]:
atlite.__version__

In [ ]:
xr.__version__

In [ ]:
%psource atlite.Cutout.convert_and_aggregate

# Desired regions


In [ ]:
desired_regions = snakemake.params.aggregated_regions
desired_regions.sort()
desired_regions

## Shape level

To get the desired data at shape level, the first input is the shapefile itself.


In [ ]:
europe = (
    gpd.read_file(snakemake.input.euroshape)
    .replace({"EL": "GR"})
    .rename(columns={"id": "index"})
    .loc[:, ["index", "geometry"]]
    .sort_values("index")
    .set_index("index")
    .drop(["AL", "IS", "CY", "MT", "ME", "MK", "NO0B1", "NO0B2", "RS", "TR", "LI"])
    .loc[desired_regions]
)

In [ ]:
europe

In [ ]:
year = snakemake.wildcards.year

In [ ]:
rectx1 = -12
rectx2 = 44
recty1 = 33
recty2 = 81
polygon = Polygon(
    [
        (rectx1, recty1),
        (rectx1, recty2),
        (rectx2, recty2),
        (rectx2, recty1),
        (rectx1, recty1),
    ]
)
europe = gpd.clip(europe, polygon)

In [ ]:
europe.plot(figsize=(15, 15));

In [ ]:
cutout = atlite.Cutout(path=snakemake.input.weatherdata, chunks={"time": 2000})

In [ ]:
cutout.prepare()

In [ ]:
europe = europe.sort_index()

# Hydro


In [ ]:
fn = snakemake.input.eiahydrogen

In [ ]:
# in billion KWh/a = TWh/a
eia_hydro_gen = (
    pd.read_csv(fn, skiprows=4, index_col=1, na_values=[" ", "--"])
    .drop(["Unnamed: 0", "Unnamed: 2"], axis=1)
    .dropna(how="all")
)
eia_hydro_gen

In [ ]:
import pycountry

In [ ]:
countries_iso2c = []
for country in eia_hydro_gen.index.values:
    try:
        countries_iso2c.append(pycountry.countries.get(name=country).alpha_2)
    except AttributeError:
        countries_iso2c.append(country)

In [ ]:
eia_hydro_gen.index = pd.Index(countries_iso2c, name="countries")
eia_hydro_gen

In [ ]:
eia_hydro_gen.rename(
    index={"Kosovo": "KV", "GB": "UK", "Czech Republic": "CZ", "Macedonia": "MK"},
    inplace=True,
)
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen.T
eia_hydro_gen

In [ ]:
eia_hydro_gen = eia_hydro_gen * 1e6  # in MWh/a
eia_hydro_gen

In [ ]:
europe.index

In [ ]:
eia_stats = eia_hydro_gen.reindex(columns=europe.index)
eia_stats

In [ ]:
eia_stats.columns

In [ ]:
europe2 = europe.copy()

In [ ]:
europe2.index.name = "countries"

In [ ]:
europe2

In [ ]:
inflow = cutout.runoff(
    shapes=europe2,
    smooth=True,
    lower_threshold_quantile=True,
    normalize_using_yearly=eia_stats,
).sel(time=slice(date_range[0], date_range[1]))

In [ ]:
# scale the inflow by installed capacities
df_installed_cap = pd.read_csv(
    snakemake.input.hydroinstalledcap,
    sep="\t",
    usecols=["Year", "AreaName", "MapCode", "ProductionType", "highRES_Europe"],
)
df_installed_cap = df_installed_cap.loc[
    df_installed_cap["ProductionType"] != "Hydro Pumped Storage"
]

In [ ]:
df_installed_cap.MapCode.unique()

In [ ]:
df_cap_shares = (
    df_installed_cap.loc[:, ["MapCode", "ProductionType", "highRES_Europe"]]
    .set_index(["ProductionType", "MapCode"])
    .unstack("ProductionType")
    .loc[:, "highRES_Europe"]
    .assign(
        Hydro=lambda x: x["Hydro Run-of-river and poundage"]
        + x["Hydro Water Reservoir"],
        share_ror=lambda x: x["Hydro Run-of-river and poundage"] / x["Hydro"],
        share_res=lambda x: x["Hydro Water Reservoir"] / x["Hydro"],
    )
    .loc[:, ["share_ror", "share_res"]]
)
df_cap_shares

In [ ]:
inflow

In [ ]:
inflow_res = (
    inflow.assign_coords(time=range(0, len(inflow.time)))
    .assign_coords(technology="HydroRes")
    .expand_dims("technology")
    .stack(final=["time", "countries", "technology"])
    .round(2)
    .to_pandas()
    .reset_index()
)

inflow_res.columns = inflow_res.columns.map(str)
inflow_res = inflow_res.rename(columns={"0": "value"})

In [ ]:
df_cap_shares.loc[:, ["share_res"]]

In [ ]:
inflow_res

In [ ]:
df_cap_shares.loc[inflow_res.countries.unique(), :]

In [ ]:
(
    inflow_res.set_index(["countries", "time", "technology"])
    .mul(
        df_cap_shares.rename(columns={"share_res": "value"}),
        axis="index",
        level="countries",
    )
    .drop(columns={"share_ror"})
    .reset_index()
    .set_index("time")
    .round(0)
    .to_csv(snakemake.output["hydroresinfl"])
)

## ROR capfac


In [ ]:
df_installed_cap.MapCode.unique()

In [ ]:
df_installed_ror = (
    df_installed_cap.loc[
        df_installed_cap["ProductionType"] == "Hydro Run-of-river and poundage"
    ]
    .set_index("MapCode")
    .drop(columns={"Year", "AreaName", "ProductionType"})
    .loc[:, "highRES_Europe"]
)

In [ ]:
df_installed_ror = df_installed_ror[df_installed_ror > 0]

In [ ]:
df_installed_ror

In [ ]:
ror_capfac = (
    inflow.to_pandas()
    .T.stack()
    .to_frame()
    .rename(columns={0: "value"})
    .mul(
        (
            df_cap_shares.reset_index()
            .rename(columns={"MapCode": "countries", "share_ror": "value"})
            .set_index("countries")
            .loc[:, ["value"]]
        )
    )
    .div(df_installed_ror, axis="index", level="countries")
    .loc[:, "value"]
)

In [ ]:
ror_capfac

In [ ]:
ror_capfac = ror_capfac.dropna()

In [ ]:
ror_capfac = ror_capfac.where(ror_capfac <= 1, 1)

In [ ]:
ror_capfac.unstack().T.plot(subplots=True, figsize=(10, 50), ylim=(0, 1.1))

In [ ]:
capacity_factor_hydro_ror = ror_capfac.to_xarray().rename({"countries": "index"})

In [ ]:
capacity_factor_hydro_ror

In [ ]:
capacity_factor_hydro_ror.name = "HydroRoR"

In [ ]:
capacity_factor_hydro_ror.time.size

In [ ]:
capacity_factor_hydro_ror.assign_coords(
    technology=lambda x: x.name, time=range(0, capacity_factor_hydro_ror.time.size)
).expand_dims("technology").stack(final=["time", "technology", "index"]).round(
    2
).to_pandas().to_csv(snakemake.output["hydrororcapfac"], header=False)